In [1]:
from langchain.document_loaders import PyPDFLoader

# PDF 가져오기
loaders = [
    PyPDFLoader("{path}/차세대 한국형 스마트팜 개발.pdf"),
    PyPDFLoader("{path}/차세대 한국형 스마트팜 개발.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [2]:
# Splitter를 이용한 청킹
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

#splitter를 이용한 문서 청킹
splits = text_splitter.split_documents(docs)
print(splits)

[Document(metadata={'source': '{path}/차세대 한국형 스마트팜 개발.pdf', 'page': 0, 'page_label': '1'}, page_content='차세대\xa0한국형스마트팜\xa0개발\n국립농업과학원\xa0농업공학부\xa0스마트팜개발과\xa0이현동\xa0연구관\n한국형\xa0스마트팜\xa0개발\xa0프로젝트\xa0!\n우리나라\xa0농업은\xa0고령화와\xa0높은\xa0노동강도로\xa0인해\xa0청년인구의\xa0유입이\xa0어려운\xa0구조로\xa0타\xa0분야보다\xa0생산인구\xa0절벽화가\xa0더욱\n가시화되고\xa0있다.\xa0이에\xa0대한\xa0대안으로\xa0제시되고\xa0있는\xa0스마트팜은\xa0사물인터넷,\xa0빅데이터,\xa0인공지능,\xa0로봇\xa0등을\xa0활용하여\n농산물의\xa0생육환경을\xa0최적상태로\xa0관리하고\xa0노동력\xa0절감과\xa0생산성\xa0향상을\xa0구현하는\xa0효율적인\xa0농업형태이다.\xa0\n차세대\xa0한국형\xa0스마트팜\xa0기술개발\xa0프로젝트는\xa04기관\xa019개\xa0전담부서가\xa0협업하여\xa0핵심\xa0요소\xa0및\xa0원천\xa0기반기술의\xa0확보를\xa0위해\n연구\xa0역량을\xa0집중하고\xa0있고\xa0국내\xa0농업여건에\xa0적합하게\xa0기술수준별로\xa0스마트팜\xa0모델을\xa03가지\xa0단계로\xa0구분하여\xa0개발을\n추진하고\xa0있다.\xa0단계별\xa0스마트팜은\xa01세대(편리성\xa0증진),\xa02세대(생산성\xa0향상-네덜란드추격형),\xa03세대(글로벌산업화-플랜트\n수출형)로\xa0구분되고\xa0기술의\xa0단계적\xa0개발과\xa0실용화\xa0계획을\xa0통해\xa0노동력과\xa0농자재의\xa0사용을\xa0줄이고,\xa0생산성과\xa0품질을\n제고함으로\xa0농가소득과\xa0연계하며,\xa0나아가\xa0영농현장의\xa0애로와\xa0연관\xa0산업의\xa0문제를\xa0동시에\xa0해결해\xa0간

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\user\AppData\Local\Temp\ipykernel_9348\370139595.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\user\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#크로마 벡터 영구 저장소 지정
persist_directory = '{path}/rag/chroma_db'

#벡터 저장소
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

16


In [5]:
question = "한국형 스마트팜이란?"

docs = vectordb.similarity_search(question,k=3)  #return 받고자하는 문서의 수를 3개로 지정

# 문서 길이 확인
print(len(docs))

# 첫번째 문서의 콘텐츠 확인
print(docs[0].page_content)

# 영구 저장
vectordb.persist()

3
차세대 한국형스마트팜 개발
국립농업과학원 농업공학부 스마트팜개발과 이현동 연구관
한국형 스마트팜 개발 프로젝트 !
우리나라 농업은 고령화와 높은 노동강도로 인해 청년인구의 유입이 어려운 구조로 타 분야보다 생산인구 절벽화가 더욱
가시화되고 있다. 이에 대한 대안으로 제시되고 있는 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여
농산물의 생육환경을 최적상태로 관리하고 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태이다. 
차세대 한국형 스마트팜 기술개발 프로젝트는 4기관 19개 전담부서가 협업하여 핵심 요소 및 원천 기반기술의 확보를 위해
연구 역량을 집중하고 있고 국내 농업여건에 적합하게 기술수준별로 스마트팜 모델을 3가지 단계로 구분하여 개발을
추진하고 있다. 단계별 스마트팜은 1세대(편리성 증진), 2세대(생산성 향상-네덜란드추격형), 3세대(글로벌산업화-플랜트
수출형)로 구분되고 기술의 단계적 개발과 실용화 계획을 통해 노동력과 농자재의 사용을 줄이고, 생산성과 품질을
제고함으로 농가소득과 연계하며, 나아가 영농현장의 애로와 연관 산업의 문제를 동시에 해결해 간다는 계획이다.
차세대 한국형 스마트팜 융복합 프로젝트
세대별 스마트팜 모델의 특징은 ?
차세대 한국형 스마트팜 프로젝트는 외국의 선진 기술을 그대로 수입・적용하거나 단순 모방하지 않고 우리 농업 환경과
현장 상황에 적합한 기술을 개발하여 농업선진국과 경쟁할 수 있는 독자적인 농업생산기술을 확보하는 것이다.
1세대 스마트팜 - 원격 모니터링과 제어로 편의성 향상
원격 관리를 이용한 농업인 편리성 증대를 목적으로 현재까지 개발한 자동화 및 ICT 기술들을 영농여건과 농가수준에 맞추어 기본형과
선택형으로 구분해 모델을 제시함으로써 농가의 필요에 따라 조건에 맞는 모델을 적용할 수 있도록 한 것이다.
모델유형 주요기능 모델유형 주요기능 모델유형 주요기능
1세대 스마트온실 모델 4종(기본형1, 선택형3)


C:\Users\user\AppData\Local\Temp\ipykernel_9348\451014003.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [6]:
question = "필요한 ICT 기술은?"

docs = vectordb.similarity_search(question,k=5) #return 받고자하는 문서의 수를 5개로 지정

# 처음 2개의 결과 확인
print(docs[0])
print('\n\n', docs[1])

page_content='제어기 등 ICT 기기 산업을 활성화 시키는데 필수적이다.
ICT 핵심부품 및 기기 표준화
 스마트온실 단체표준 제정
파이팅! 한국형 스마트팜
사회 전반에서 4차 산업 혁명에 대한 많은 관심을 가지고 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하는 분야를
확장하고 있다. 농업분야에서도 스마트 기기를 농사에 효과적으로 접목할 수 있는 ICT분야 연구개발이 필요하다.
첨단화되는 스마트 기기와 다양한 실시간 정보를 활용해 편리성과 생산성을 높이는 스마트팜 연구개발은 국내농가의
소득증대뿐만 아니라 농업 생산기술과 시스템 수출의 길을 열고, 우리 농업기술의 국제경쟁력을 높이는 데에도 크게 기여할
것으로 기대 된다. 가까운 장래에 한국형 스마트 팜의 기반이 구축되고 우리 경제의 미래 성장 동력으로 자리매김 할 우리
농업의 모습을 기대해 본다.' metadata={'page': 3, 'page_label': '4', 'source': '{path}/rag/documents/차세대 한국형 스마트팜 개발.pdf'}


 page_content='제어기 등 ICT 기기 산업을 활성화 시키는데 필수적이다.
ICT 핵심부품 및 기기 표준화
 스마트온실 단체표준 제정
파이팅! 한국형 스마트팜
사회 전반에서 4차 산업 혁명에 대한 많은 관심을 가지고 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하는 분야를
확장하고 있다. 농업분야에서도 스마트 기기를 농사에 효과적으로 접목할 수 있는 ICT분야 연구개발이 필요하다.
첨단화되는 스마트 기기와 다양한 실시간 정보를 활용해 편리성과 생산성을 높이는 스마트팜 연구개발은 국내농가의
소득증대뿐만 아니라 농업 생산기술과 시스템 수출의 길을 열고, 우리 농업기술의 국제경쟁력을 높이는 데에도 크게 기여할
것으로 기대 된다. 가까운 장래에 한국형 스마트 팜의 기반이 구축되고 우리 경제의 미래 성장 동력으로 자리매김 할 우리
농업의 모습을 기대해 본다.' metadata={'page': 3, 'page_label': '4